In [165]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn.model_selection as skl
import sklearn.metrics as sklm
from sklearn.metrics import f1_score

! pip install pymorphy2
import pymorphy2 # Морфологический анализатор

import string # for work with strings
import nltk   # Natural Language Toolkit
from nltk.stem.snowball import SnowballStemmer 

# classifiers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [166]:
train_df = pd.read_csv("data/train.csv")

train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [167]:
test_df = pd.read_csv("data/test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [168]:
train_df["target"].value_counts()

False    118594
True      16715
Name: target, dtype: int64

In [169]:
X_train = train_df["title"].values
X_test = test_df["title"].values

y_train = train_df["target"].astype(int).values

# Decision TREE classifier

In [97]:
vectorizer = CountVectorizer()

clf = DecisionTreeClassifier(max_depth=50)

In [98]:
df_X_train_vectorized = vectorizer.fit_transform(X_train)

In [99]:
%%time
clf.fit(df_X_train_vectorized, y_train)

Wall time: 3.73 s


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=50,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [100]:
y_pred = clf.predict(df_X_train_vectorized)

In [101]:
f1_score(y_train, y_pred)

0.9168770635074771

### Submit TREE

In [104]:
df_X_test_vectorized = vectorizer.transform(X_test)

test_df["target"] = clf.predict(df_X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("data/ml_baseline_tree.csv", index=False)

!cat ml_baseline.csv | head

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


## Logistic Regression Classifier

In [218]:
vectorizer = CountVectorizer(analyzer='word')
clf = LogisticRegression()

#### data cleaning

In [219]:
word_tokenizer = nltk.WordPunctTokenizer()

# get russian and english stop words
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')
stop_words.append(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\DG
[nltk_data]     Coach\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [220]:
def process_data(texts):
    updated_texts = []
    
    # string processers
    stemmer = SnowballStemmer('russian')
    morph = pymorphy2.MorphAnalyzer()
    
    for text in texts:    
        text_lower = text.lower() # convert words in a text to lower case
        tokens     = word_tokenizer.tokenize(text_lower) # splits the text into tokens (words)
        
        # remove punct and stop words from tokens
        tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words)]
        
        tmp_str = ''
        for token in tokens:
            # not taking in account 0 or 1 letter tokens
            if (len(token) >= 2):
                # lemmatization + stemming
                token = (morph.parse(token))[0].normal_form
                token = stemmer.stem(token)
                
                tmp_str += token + ' '
        
        updated_texts.append(tmp_str.strip())
    
    return updated_texts

In [203]:
print(X_train)
texts = process_data(X_train)
print(texts[0:10])

['Экс-министр экономики Молдовы - главе МИДЭИ, цель которого сделать из республики не просителя, а донора: Надо избегать долгого нахождения н�'
 'Эта песня стала известна многим телезрителям благодаря сериалу Диверсант-2'
 'Банши 4 сезон 2 серия Бремя красоты смотреть онлайн!' ...
 '1941 Plymouth Special Deluxe Hot Rod, Automatic,Small Block,Power Steering, A/C for sale: photos, technical specifications, description'
 "Купить It's Skin Сыворотка питательная Power 10 Formula YE Effector, 30 мл в Москве - WOWCREAM"
 'Технический спирт в канистрах и флаконах, купить в Москве']
['экс министр экономик молдов глав мидэ цел котор сдела республик просител донор избега долг нахожден', 'этот песн стат известн мног телезрител благодар сериа диверсант', 'банш сезон сер брем красот смотрет онлайн', 'бес картинк', 'новомосковск сыгра следж хоккеист алексинск звезд сборн кита ...', 'салат корейск морков копчен куриц кукуруз сухарик калорийн белк жир углевод', 'углов пневмогайковерт fubag rwc 105 1003

### Splitting and fitting

In [221]:
# splitting using function
df_X_train, df_X_test, y_tra, y_tes = skl.train_test_split(texts, y_train, test_size=0.2, random_state=42)

In [222]:
df_X_train_vectorized = vectorizer.fit_transform(df_X_train)

In [223]:
%%time
clf.fit(df_X_train_vectorized, y_tra)

print(clf.n_iter_)

[14]
Wall time: 1.04 s


In [224]:
y_pred = clf.predict(df_X_train_vectorized)

In [225]:
f1_score(y_tra, y_pred)

# -12 = 0.9749051291467744

0.9737959527179526

### Submit Logistic

In [233]:
df_X_train_vectorized = vectorizer.fit_transform(texts)

In [234]:
%%time
clf.fit(df_X_train_vectorized, y_train)

print(clf.n_iter_)

[12]
Wall time: 1.42 s


In [237]:
df_X_test_vectorized = vectorizer.transform(X_test)

In [238]:
test_df["target"] = clf.predict(df_X_test_vectorized).astype(bool)

test_df[["id", "target"]].to_csv("data/ml_baseline_logistic.csv", index=False)

!cat ml_baseline.csv | head

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
